In [1]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [2]:
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import colors as col
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import interact, fixed

In [5]:
def plot(x, y, water, Timestep, save=False):
    mask = water[:,:,Timestep] < 0.00000001
    layer = water[:,:,Timestep]
#     layer[mask] = None
    
    water_cm = col.LinearSegmentedColormap.from_list(
        'water',
        [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
    )
    
    fig = plt.figure(figsize=(16,8))
    
    ax1 = fig.add_subplot(122)
    ax1.imshow(layer, cmap='viridis')#, vmin=0, vmax=np.max(water))
    
    ax2 = fig.add_subplot(121, projection='3d')
    ax2.plot_surface(y, x, layer, cmap=water_cm)
    ax2.set_zlim(0, 5)
    
    if save:
        plt.savefig('shallow/sw-{:05}.png'.format(save))
        fig.clf()
    else:
        plt.show()

In [82]:
size_x = 50
size_y = 50
size_t = 1000

height = np.zeros((size_x, size_y, size_t))
flow_u = np.zeros((size_x, size_y, size_t))
flow_v = np.zeros((size_x, size_y, size_t))

limit = 3
x, y = np.meshgrid(np.linspace(0, limit, size_x), np.linspace(0, limit, size_y))

func = lambda x, y, mu, simga: np.e ** -(((x-mu)/simga)**2 + ((y-mu)/simga)**2)

amplitude = limit
mu = 0
sigma = limit/10

height[:,:,0] = amplitude * func(x, y, mu, sigma)
height[:,:,1] = amplitude * func(x, y, mu, sigma)

c1 = 0.1
c2 = 0.1

for i in log_progress(range(2, size_t), every=1, name='Timestep'):
    # Center++
    height[1:-1,1:-1,i] = height[1:-1,1:-1,i-2] - \
        c1 * (flow_u[2:,1:-1,i-1] - flow_u[:-2,1:-1,i-1]) - \
        c2 * (flow_v[1:-1,2:,i-1] - flow_v[1:-1,:-2,i-1])
    
    flow_u[1:-1,:,i] = flow_u[1:-1,:,i-2] - \
        c1 * (height[2:,:,i-1] - height[:-2,:,i-1])
    
    flow_v[:,1:-1,i] = flow_v[:,1:-1,i-2] - \
        c2 * (height[:,2:,i-1] - height[:,:-2,i-1])
    
    # x-Edges
    height[0,1:-1,i] = height[0,1:-1,i-2] - \
        c1 * (- flow_u[2,1:-1,i-1] + 4*flow_u[1,1:-1,i-1] - 3*flow_u[0,1:-1,i-1]) - \
        c2 * (flow_v[0,2:,i-1] - flow_v[0,:-2,i-1])
    height[-1,1:-1,i] = height[-1,1:-1,i-2] - \
        c1 * (flow_u[-3,1:-1,i-1] - 4*flow_u[-2,1:-1,i-1] + 3*flow_u[-1,1:-1,i-1]) - \
        c2 * (flow_v[-1,2:,i-1] - flow_v[-1,:-2,i-1])
    
    flow_u[0,:,i] = flow_u[0,:,i-2] - \
        c1 * (- height[2,:,i-1] + 4*height[1,:,i-1] - 3*height[0,:,i-1])
    flow_u[-1,:,i] = flow_u[-1,:,i-2] - \
        c1 * (height[-3,:,i-1] - 4*height[-2,:,i-1] + 3*height[-1,:,i-1])
    
    # y-Edges
    height[1:-1,0,i] = height[1:-1,0,i-2] - \
        c1 * (flow_u[2:,0,i-1] - flow_u[:-2,0,i-1]) - \
        c2 * (- flow_v[1:-1,2,i-1] + 4*flow_v[1:-1,1,i-1] - 3*flow_v[1:-1,0,i-1])
    height[1:-1,-1,i] = height[1:-1,-1,i-2] - \
        c1 * (flow_u[2:,-1,i-1] - flow_u[:-2,-1,i-1]) - \
        c2 * (flow_v[1:-1,-3,i-1] - 4*flow_v[1:-1,-2,i-1] + 3*flow_v[1:-1,-1,i-1])
    
    flow_v[:,0,i] = flow_v[:,0,i-2] - \
        c2 * (- height[:,2,i-1] + 4*height[:,1,i-1] - 3*height[:,0,i-1])
    flow_v[:,-1,i] = flow_v[:,-1,i-2] - \
        c2 * (height[-3,:,i-1] - 4*height[-2,:,i-1] + 3*height[-1,:,i-1])
    
    # Corners
    height[0,0,i] = height[0,0,i-2] - \
        c1 * (- flow_u[2,0,i-1] + 4*flow_u[1,0,i-1] - 3*flow_u[0,0,i-1]) - \
        c2 * (- flow_v[0,2,i-1] + 4*flow_v[0,1,i-1] - 3*flow_v[0,0,i-1])
    height[-1,0,i] = height[-1,0,i-2] - \
        c1 * (flow_u[-3,0,i-1] - 4*flow_u[-2,0,i-1] + 3*flow_u[-1,0,i-1]) - \
        c2 * (- flow_v[-1,2,i-1] + 4*flow_v[-1,1,i-1] - 3*flow_v[-1,0,i-1])
    height[0,-1,i] = height[0,-1,i-2] - \
        c1 * (- flow_u[2,-1,i-1] + 4*flow_u[1,-1,i-1] - 3*flow_u[0,-1,i-1]) - \
        c2 * (flow_v[0,-3,i-1] - 4*flow_v[0,-2,i-1] + 3*flow_v[0,-1,i-1])
    height[-1,-1,i] = height[-1,-1,i-2] - \
        c1 * (flow_u[-3,-1,i-1] - 4*flow_u[-2,-1,i-1] + 3*flow_u[-1,-1,i-1]) - \
        c2 * (flow_v[-1,-3,i-1] - 4*flow_v[-1,-2,i-1] + 3*flow_v[-1,-1,i-1])

In [83]:
interact(plot, x=fixed(x), y=fixed(y), water=fixed(flow_u), Timestep=(0, size_t-1, 10))

interactive(children=(IntSlider(value=490, description='Timestep', max=999, step=10), Output()), _dom_classes=…

<function __main__.plot(x, y, water, Timestep)>

---

In [8]:
size_x = 50
size_y = 50
size_t = 1000

height = np.zeros((size_x, size_y, size_t))
flow_u = np.zeros((size_x, size_y, size_t))
flow_v = np.zeros((size_x, size_y, size_t))

limit = 3
x, y = np.meshgrid(np.linspace(0, limit, size_x), np.linspace(0, limit, size_y))

func = lambda x, y, mu, simga: np.e ** -(((y-mu)/simga)**2)

amplitude = limit
mu = limit/4
sigma = limit/10

height[:,:,0] = amplitude * func(x, y, mu, sigma)
height[:,:,1] = amplitude * func(x, y, mu, sigma)

c1 = 0.1
c2 = 0.1

for i in log_progress(range(2, size_t), every=1, name='Timestep'):
    # Center++
    height[1:-1,1:-1,i] = height[1:-1,1:-1,i-2] - \
        c1 * (flow_u[2:,1:-1,i-1] - flow_u[:-2,1:-1,i-1]) - \
        c2 * (flow_v[1:-1,2:,i-1] - flow_v[1:-1,:-2,i-1])
    
    flow_u[1:-1,:,i] = flow_u[1:-1,:,i-2] - \
        c1 * (height[2:,:,i-1] - height[:-2,:,i-1])
    
    flow_v[:,1:-1,i] = flow_v[:,1:-1,i-2] - \
        c2 * (height[:,2:,i-1] - height[:,:-2,i-1])
    
    # x-Edges
    height[0,1:-1,i] = height[0,1:-1,i-2] - \
        c1 * (flow_u[1,1:-1,i-1] - flow_u[-1,1:-1,i-1]) - \
        c2 * (flow_v[0,2:,i-1] - flow_v[0,:-2,i-1])
    height[-1,1:-1,i] = height[-1,1:-1,i-2] - \
        c1 * (flow_u[0,1:-1,i-1] - flow_u[-2,1:-1,i-1]) - \
        c2 * (flow_v[-1,2:,i-1] - flow_v[-1,:-2,i-1])
    
    flow_u[0,:,i] = flow_u[0,:,i-2] - \
        c1 * (height[1,:,i-1] - height[-1,:,i-1])
    flow_u[-1,:,i] = flow_u[-1,:,i-2] - \
        c1 * (height[0,:,i-1] - height[-2,:,i-1])
    
    # y-Edges
    height[1:-1,0,i] = height[1:-1,0,i-2] - \
        c1 * (flow_u[2:,0,i-1] - flow_u[:-2,0,i-1]) - \
        c2 * (flow_v[1:-1,1,i-1] - flow_v[1:-1,-1,i-1])
    height[1:-1,-1,i] = height[1:-1,-1,i-2] - \
        c1 * (flow_u[2:,-1,i-1] - flow_u[:-2,-1,i-1]) - \
        c2 * (flow_v[1:-1,0,i-1] - flow_v[1:-1,-2,i-1])
    
    flow_v[:,0,i] = flow_v[:,0,i-2] - \
        c2 * (height[:,1,i-1] - height[:,-1,i-1])
    flow_v[:,-1,i] = flow_v[:,-1,i-2] - \
        c2 * (height[:,0,i-1] - height[:,-2,i-1])
    
    # Corners
    height[0,0,i] = height[0,0,i-2] - \
        c1 * (flow_u[1,0,i-1] - flow_u[-1,0,i-1]) - \
        c2 * (flow_v[0,1,i-1] - flow_v[0,-1,i-1])
    height[-1,0,i] = height[-1,0,i-2] - \
        c1 * (flow_u[0,0,i-1] - flow_u[-2,0,i-1]) - \
        c2 * (flow_v[-1,1,i-1] - flow_v[-1,-1,i-1])
    height[0,-1,i] = height[0,-1,i-2] - \
        c1 * (flow_u[1,-1,i-1] - flow_u[-1,-1,i-1]) - \
        c2 * (flow_v[0,0,i-1] - flow_v[0,-2,i-1])
    height[-1,-1,i] = height[-1,-1,i-2] - \
        c1 * (flow_u[0,-1,i-1] - flow_u[-2,-1,i-1]) - \
        c2 * (flow_v[-1,0,i-1] - flow_v[-1,-2,i-1])

In [9]:
interact(plot, x=fixed(x), y=fixed(y), water=fixed(height), Timestep=(0, size_t-1, 10))

interactive(children=(IntSlider(value=490, description='Timestep', max=999, step=10), Checkbox(value=False, de…

<function __main__.plot(x, y, water, Timestep, save=False)>

In [97]:
height.shape[2]

1000

In [10]:
for i in range(height.shape[2]):
    plot(x, y, height, i, i+1)

/home/vlizana/sci/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

...

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>

<Figure size 1152x576 with 0 Axes>